In [3]:
import requests
import urllib3


from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

In [4]:
# Custom SSL context to allow smaller DH keys
class SSLAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        context.set_ciphers('DEFAULT@SECLEVEL=1')  # Lowering the security level
        self.poolmanager = urllib3.PoolManager(*args, ssl_context=context, **kwargs)

In [5]:
# Fectch the website u want
def fetch_web_page(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    headers = {'User-Agent': user_agent}

    # Create a session and mount the custom SSL adapter
    session = requests.Session()
    session.mount('https://', SSLAdapter())

    # Make the request with headers
    response = session.get(url, headers=headers)
    response.encoding = 'utf-8'
    print(response)

    # Parse the response content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [6]:
# Fetch all the courses' url of a department
def fetch_courses_url(url):
    # Fetching all the courses' name & url
    iim_soup = fetch_web_page(url)
    # Find all td elements containing course information
    course_blocks = iim_soup.find_all('td')

    courses_url = []
    #courses_info = []  # 改动：将原本的 courses_url 改为 courses_info，包含课程名称和 URL
    course_list = []
    for block in course_blocks:
        if block.find('span', class_='course_name'):
            course_name = block.find('span', class_='course_name').getText().strip()
            course_url = ""
            if block.find('a'):  # Directly look for 'a' tag under 'block'
                a_tag = block.find('a')
                course_url = a_tag['href'].strip()
            # Check if the course name is already added to avoid duplicates
            if course_name and course_name not in course_list:
                course_list.append(course_name)
                if course_url: 
                    #courses_info.append((course_name, course_url))  # 改动：将课程名称和 URL 一起添加
                    courses_url.append(course_url)
                    #courses_info.append((course_name, course_url))  # 改动：将课程名称和 URL 一起添加
                print("Course Name:", course_name)
                print("Course Url:", course_url)
                print()

    print(course_list)
    return courses_url
    

In [7]:
def fectch_syllabus_url(url):
    course_soup = fetch_web_page(url)
    course_elements = course_soup.find_all('td')
    newest_syllabus_url = course_elements[2].find('a')['href'].strip()

    return newest_syllabus_url

In [16]:
# Fetch the syllabus url and content
def fetch_syllabus_page(url):
    
    syllabus_soup = fetch_web_page(url)
    syllabus_content = syllabus_soup.find('div', id = 'content')
    header_div = syllabus_soup.find('div', id='header')

    # 找到header_div中的所有span元素
    span_elements = header_div.find_all('span')

    # 选择列表中的第二个span元素
    second_span_element = span_elements[1]

    # 找到second_span_element中的br元素的下一个兄弟节点，即br后面的文本内容
    course_name = second_span_element.find('br').next_sibling.strip()

    syllabus_div_elements = syllabus_content.find_all('div', class_ = 'block')
    syllabus_description = syllabus_div_elements[2].getText().strip()
    syllabus_progress = syllabus_div_elements[4]
    #print(course_name)
    print(syllabus_description)
    print('----------------------')
    print()

    side_bar_elements = syllabus_soup.find('div', id = 'sidebar')
    grading_div = side_bar_elements.find_all('div', class_='block')[6]
    teaching_strats_div = side_bar_elements.find_all('div', class_='block')[7]

    # def fetch_table_data(div):
    #     table = div.find('table')
    #     if table :    
    #         tbody = table.find('tbody')
    #         headers = []
    #         for th in tbody.find_all('th'):
    #             headers.append(th.text.strip())
    #         rows = []
    #         for tr in tbody.find_all('tr'):
    #             row = []
    #             # 处理th元素
    #             for th in tr.find_all('th'):
    #                 row.append(th.text.strip())
    #             # 处理td元素
    #             for td in tr.find_all('td'):
    #                 row.append(td.text.strip())
    #                 print(td.text.strip())
    #             rows.append(row)
    #     else:
    #         print("error")

    #     print('----------------------')
    def fetch_table_data(div):
        table = div.find('table')
        if table:    
            tbody = table.find('tbody')
            headers = []
            for th in tbody.find_all('th'):
                headers.append(th.text.strip())
            rows = []
            for tr in tbody.find_all('tr'):
                row = []
                # 处理th元素
                for th in tr.find_all('th'):
                    row.append(th.text.strip())
                # 处理td元素
                for td in tr.find_all('td'):
                    row.append(td.text.strip())
                rows.append(row)
            return headers, rows
        else:
            print("error")

    # 调用函数获取数据
    
    syllabus_progress_headers = None
    syllabus_progress_rows = None
    teaching_strats_headers = None
    teaching_strats_rows = None
    grading_headers = None 
    grading_rows = None
    if fetch_table_data(grading_div):
        grading_headers, grading_rows = fetch_table_data(grading_div)
    if fetch_table_data(teaching_strats_div):
        teaching_strats_headers, teaching_strats_rows = fetch_table_data(teaching_strats_div)
    if fetch_table_data(syllabus_progress):
        syllabus_progress_headers, syllabus_progress_rows = fetch_table_data(syllabus_progress)
    
    
    #if grading_div and teaching_strats_div and syllabus_progress:  # 添加此行
    # 将数据保存到文本文件
        with open('syllabus_data.txt', 'a', encoding='utf-8') as file:
            file.write(course_name + '\n\n')

            # 写入评分表格数据
            file.write("Grading:\n")
            if grading_headers and grading_rows:
                file.write('\t'.join(grading_headers) + '\n')
                for row in grading_rows:
                    file.write('\t'.join(row) + '\n')
                file.write('\n')
            
            # 写入教学策略表格数据
            file.write("Teaching Strategies:\n")
            if teaching_strats_headers and teaching_strats_rows:
                file.write('\t'.join(teaching_strats_headers) + '\n')
                for row in teaching_strats_rows:
                    file.write('\t'.join(row) + '\n')
                file.write('\n')

            # 写入大纲进度表格数据
            file.write("Syllabus Progress:\n")
            if syllabus_progress_headers and syllabus_progress_rows:
                file.write('\t'.join(syllabus_progress_headers) + '\n')
                for row in syllabus_progress_rows:
                    file.write('\t'.join(row) + '\n')


In [15]:
fetch_syllabus_page("https://class-qry.acad.ncku.edu.tw/syllabus/online_display.php?syear=0112&sem=2&co_no=J030800&class_code=")

<Response [200]>
課程概述 Course Description
       隨著科技的進步，電腦運算量得以大幅提升，人工智慧運算被視為用來解決人類問題的方式之一，學習神經元的神經網路運作模式，新興記憶體元件能夠模仿神經元的傳遞方式與運作方法，能夠作為記憶體運算以及類神經網路運算的基本單元。因此，本課程將深入介紹各種記憶體、類神經網路以及兩者如何相互結合來達到人工智慧運算。Today, computing power drastically increases as technology evolves. AI is considered as one of possible solutions to the problems that human beings are facing now. In addition, emerging memories have some important features that can emulate the working mechanisms between neurons and synapses. Hence, they can be the building blocks for both computing-in-memory and neural networks. So, in this course, students will learn memory technology, neural networks and the combination of these two technologies to achieve AI.
----------------------



In [17]:
iim_course_url = 'https://course.ncku.edu.tw/index.php?c=qry11215&i=UDUFbVQxUjIKJQN1B2sLPAFsAX8ANwMjUW9VagBnVjcOPgM2U2dQdgM9VjNTNVt3XWpRJQY1B2BTYgdgADJeIVc6AGgAYwZsUDRRb1ViADwBZ1cxAyZVJFFuD24EY1MiDmgAZAd4ACEOVQE7UmEALAVqUXRUZgUxBTRUIFoTV2pQdgVsVHhSdQotAzwHYAs3AXQBLwB8AzpRNVU5AG5WPw53A2tTZlA1AzxWa1M1W2VdMlFuBjQHMFNjBzgAMl5oV2MAYgBqBjVQPlFmVTgAbwFsVzkDbFVkUW8PJAQqUzoOPQA7ByoAMg4oAW5SbAA0BWFRPFQh'
courses_url = fetch_courses_url(iim_course_url)
print(courses_url)
for url in courses_url:
    syllabus_url = fectch_syllabus_url(url)
    fetch_syllabus_page(syllabus_url)

<Response [200]>
Course Name: 流行樂賞析與實務
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A92F300

Course Name: 音樂文化導引
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A92F100

Course Name: 批判思考:當代文學理論
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A937200

Course Name: 戲劇與電影
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A92E800

Course Name: 表演藝術與實作
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A93A200

Course Name: 戲劇賞析
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A92D300

Course Name: 朝聖的世界史
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A94C200

Course Name: 傳統戲曲與表演藝術
Course Url: https://class-qry.acad.ncku.edu.tw/crm/course_map/course.php?dept=A9&cono=A93A300

Course Name: 藝術、設計與生活
Course Url: https://class-qry.aca